In [2]:
import pandas as pd

df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])
 

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])
 

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                     'B': ['B8', 'B9', 'B10', 'B11'],
                     'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])

result = pd.concat([df1, df2, df3])

print (result)

      A    B    C    D
0    A0   B0   C0   D0
1    A1   B1   C1   D1
2    A2   B2   C2   D2
3    A3   B3   C3   D3
4    A4   B4   C4   D4
5    A5   B5   C5   D5
6    A6   B6   C6   D6
7    A7   B7   C7   D7
8    A8   B8   C8   D8
9    A9   B9   C9   D9
10  A10  B10  C10  D10
11  A11  B11  C11  D11


# Shapiro Wilk and Anderson Darling test (for Normality)

In [3]:
import numpy as np
from scipy import stats
X = np.random.normal (10,5, 200)
Shapiro_value, p_value = stats.shapiro(X)
print (Shapiro_value, p_value)

### Can not reject the hypothesis that 'the sampled data is normally distributed due to high p value (>=0.05)'

anderson_darling, p_value, significant_value = stats.anderson(X)
print (anderson_darling, p_value, significant_value)

# Fail to reject the hypothesis that 'the sampled data is normally distributed'

0.9967260360717773 0.9503976106643677
0.18076157743689691 [0.565 0.644 0.772 0.901 1.071] [15.  10.   5.   2.5  1. ]


In [7]:
X = np.random.rand (200) * 10
Shapiro_value, p_value = stats.shapiro(X)
print (Shapiro_value, p_value)

### Can reject the hypothesis that 'the sampled data is normally distributed due to low p value (5.3220974223222584e-05)'

anderson_darling, p_value, significant_value = stats.anderson(X)
print (anderson_darling, p_value, significant_value)

# Fail to reject the hypothesis that 'the sampled data is normally distributed'

anderson_darling, p_value, significant_value = stats.anderson(X, dist='gumbel')
print (anderson_darling, p_value, significant_value)

0.9542248249053955 4.9271002353634685e-06
2.3167980662400964 [0.565 0.644 0.772 0.901 1.071] [15.  10.   5.   2.5  1. ]
2.6413969373036537 [0.467 0.628 0.746 0.865 1.024] [25.  10.   5.   2.5  1. ]


# A detailed implmentation of log likelihood ratio test.

In [46]:
from scipy.stats import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

print (chi2.isf(0.05, 1))
print (chi2.sf(1.8, 1), chi2.cdf(1.8, 1))
print (chi2.sf(2.8, 1), chi2.cdf(2.8, 1))

def likelihood_ratio_test(feature_alternate, feature_null, labels):
    labels = np.array(labels)
    feature_alternate = np.array(feature_alternate)
    feature_null = np.array(feature_null)
    
    lr_model = LogisticRegression()
    
    lr_model.fit (feature_null, labels)
    null_prob_full = lr_model.predict_proba(feature_null)
    null_prob = null_prob_full[:,1]
    print (null_prob_full)
    print (null_prob)
    print (np.sum(np.log(null_prob)))
    
    print ("_____________________________________")
    
    df = feature_alternate.shape[1] - feature_null.shape[1]
    
    lr_model.fit (feature_alternate, labels)
    alt_prob_full = lr_model.predict_proba(feature_alternate)
    alt_prob = alt_prob_full [:,1]
    print (alt_prob_full)
    print (alt_prob)
    print (np.sum(alt_prob))
    
    null_log = -log_loss(labels, null_prob, normalize = False)
    alt_log = -log_loss (labels, alt_prob, normalize = False)
    
    G = 2 * (alt_log - null_log)
    p_value = chi2.sf (G, df)

    return (alt_log, null_log, G, df, p_value)
    

3.8414588206941285
0.17971249487899593 0.8202875051210001
0.09426430684121058 0.9057356931587894


In [47]:
CombData=pd.read_csv(r"C:\Learning Documents\Power Detergent Project\data1.csv").iloc[0:100]
CombData.head()

,TimeStamp,GasFlowRate_T2,Moisture,SlurryPressureJets_T2,SlurryFlowRate_T2,VacuumPressureT2,EntranceTemp_T2,HotAirFlowRate_T2,SlurryTemp_T2,ZeoliteFlowRate_T2,...,Data:,Hora,Fórmula,Density,DateTime,ProductCode,Timestamp,SlurryMoisture,GasFlowPerSlurry,HotAirPerSlurry
0,3/14/2018 10:30,882.365280,3.662309,50.713224,37.207056,-419.506311,337.261429,110.471841,106.248383,338.108892,...,14/3/18,10:30,BRILHANTE - NNOC,575.666667,3/14/2018 10:30,.67532244/8- BRI BF57.5,3/14/2018 12:00,26.48,23.714998,2.969110
1,3/14/2018 10:32,921.705371,4.023508,51.487246,37.730728,-418.732101,346.286298,110.305588,106.248383,296.495490,...,14/3/18,10:45,BRILHANTE - NNOC,NaN,3/14/2018 10:45,.67532244/8- BRI BF57.5,3/14/2018 12:00,26.48,24.428508,2.923495
2,3/14/2018 10:34,872.642339,4.041180,52.140483,37.860631,-415.378572,353.631434,110.144450,106.248383,254.882088,...,14/3/18,10:45,BRILHANTE - NNOC,NaN,3/14/2018 10:45,.67532244/8- BRI BF57.5,3/14/2018 12:00,26.48,23.048806,2.909208
3,3/14/2018 10:36,852.873065,4.355342,51.710035,37.695667,-410.176767,351.546051,110.190860,106.248383,213.268686,...,14/3/18,10:45,BRILHANTE - NNOC,NaN,3/14/2018 10:45,.67532244/8- BRI BF57.5,3/14/2018 12:00,26.48,22.625228,2.923170
4,3/14/2018 10:38,779.339708,4.630512,46.292423,32.720189,-408.399371,348.587196,107.424516,106.248383,171.655284,...,14/3/18,10:45,BRILHANTE - NNOC,NaN,3/14/2018 10:45,.67532244/8- BRI BF57.5,3/14/2018 12:00,26.48,23.818313,3.283126


In [48]:
feature_alternate = CombData.loc[:, ['GasFlowRate_T2', 'Moisture', 'SlurryPressureJets_T2', 'SlurryFlowRate_T2', 'VacuumPressureT2', 'EntranceTemp_T2', 'HotAirFlowRate_T2']]
feature_null = CombData.loc [:, ['GasFlowRate_T2', 'Moisture', 'SlurryPressureJets_T2', 'SlurryFlowRate_T2']]
labels = np.random.choice([0,1], 100)

In [49]:
alt_log, null_log, G, df, p_value = likelihood_ratio_test (feature_alternate, feature_null, labels)
print (alt_log, null_log, G, df, p_value)

[[0.50327555 0.49672445]
 [0.51513567 0.48486433]
 [0.48495031 0.51504969]
 [0.47210531 0.52789469]
 [0.49371147 0.50628853]
 [0.63201149 0.36798851]
 [0.53298072 0.46701928]
 [0.53950988 0.46049012]
 [0.51838609 0.48161391]
 [0.52474804 0.47525196]
 [0.57382664 0.42617336]
 [0.61597338 0.38402662]
 [0.59897219 0.40102781]
 [0.61170287 0.38829713]
 [0.62569588 0.37430412]
 [0.61192874 0.38807126]
 [0.61130892 0.38869108]
 [0.61289613 0.38710387]
 [0.61105578 0.38894422]
 [0.6046496  0.3953504 ]
 [0.60482075 0.39517925]
 [0.60016073 0.39983927]
 [0.52680103 0.47319897]
 [0.50698103 0.49301897]
 [0.50770479 0.49229521]
 [0.48469397 0.51530603]
 [0.48042904 0.51957096]
 [0.48218252 0.51781748]
 [0.48228137 0.51771863]
 [0.48588573 0.51411427]
 [0.47626438 0.52373562]
 [0.47481691 0.52518309]
 [0.47893979 0.52106021]
 [0.48087229 0.51912771]
 [0.4694662  0.5305338 ]
 [0.47442593 0.52557407]
 [0.47288808 0.52711192]
 [0.4752706  0.5247294 ]
 [0.47923895 0.52076105]
 [0.48004862 0.51995138]


In [50]:
result = log_loss(["spam", "ham"],  [[.1, .9], [.9, .1]])
print (result)


0.10536051565782628


In [51]:
np.log(0.1) * 0.1 + np.log (0.9) * 0.9 + np.log(0.1) * 0.1 + np.log (0.9) * 0.9 

-0.6501659467828964

In [11]:
# from sklearn.preprocessing import MinMaxScaler
# data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
# scaler = MinMaxScaler()
# print(scaler.fit(data))
# MinMaxScaler(copy=True, feature_range=(0, 1))
# print(scaler.data_max_)

# print(scaler.transform(data))

# print(scaler.transform([[2, 2]]))

from sklearn.preprocessing import MinMaxScaler
data =[[-1,2], [-0.5,6], [0,10], [1,8]]
scaler =MinMaxScaler()
# Fit must be happens before the transform.
print (scaler.fit(data))
print (scaler.get_params)
print(scaler.transform(data))
print(scaler.data_max_)

MinMaxScaler(copy=True, feature_range=(0, 1))
<bound method BaseEstimator.get_params of MinMaxScaler(copy=True, feature_range=(0, 1))>
[[0.   0.  ]
 [0.25 0.5 ]
 [0.5  1.  ]
 [1.   0.75]]
[ 1. 10.]


In [10]:
print(scaler.transform([[2, 4]]))

[[1.5  0.25]]
